<a href="https://colab.research.google.com/github/punkmic/Topic-Modeling-Reclame-Aqui/blob/master/Topic_Modeling_with_BERTopic_Reclame_aqui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Topic Modeling with BERTopic - Reclame Aqui**

BERTopic is a topic modeling technique that leverages transformers and a custom class-based TF-IDF to create dense clusters allowing for easily interpretable topics whilst keeping important words in the topic descriptions 

Reference: (https://maartengr.github.io/BERTopic/index.html).

### **Enabling the GPU**

We will use the GPU provided by COLAB to accelarate our model training. To enable GPUs for the notebook:
1- Navigate to Edit -> Notebook Settings
2- Select GPU from the Hardware Accelerator drop-down

In [187]:
# verify if GPU is enable
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Jan  5 03:46:51 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P0    33W /  70W |   3020MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### **Setup**

In [188]:
%%capture
!pip install pyspellchecker
!pip install bertopic
!pip install kaleido # for save BERTopic plots as image

In [189]:
import pandas as pd # for data manipulation
import os # for interacting with the operating system
import nltk # for natural language processing
import string # for string manipulation
import re # for for regular expressions
import matplotlib.pyplot as plt # for visualization
import spacy # for lemmatize portuguese text
from bertopic import BERTopic # for topic modeling
from spellchecker import SpellChecker # for spell check

In [190]:
# Install spacy pt_core_news_sm for portuguese text
!python -m spacy download pt_core_news_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached https://github.com/explosion/spacy-models/releases/download/pt_core_news_sm-3.4.0/pt_core_news_sm-3.4.0-py3-none-any.whl (13.0 MB)
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [191]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [192]:
# Download dataset with stopwords
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [193]:
# Donwload datasets for lemmatization
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [194]:
# Donwload dependency need to stem portuguese text
nltk.download('rslp')

[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

### **Load data from [Github](https://github.com/punkmic/Topic-Modeling-Reclame-Aqui.git)**

In [195]:
!git clone https://github.com/punkmic/Topic-Modeling-Reclame-Aqui.git

fatal: destination path 'Topic-Modeling-Reclame-Aqui' already exists and is not an empty directory.


In [196]:
# Change directory
%cd /content/Topic-Modeling-Reclame-Aqui 

# Update files from remote repository
!git pull 

# Return to work directory
%cd ..

# Check current directory
!pwd

/content/Topic-Modeling-Reclame-Aqui
Already up to date.
/content
/content


## **Prepare data**

In [197]:
def read_data(path_csv, drop_duplicates = True, lower=True):
 
  # Use the read_csv method to read csv file
  df = pd.read_csv(path_csv)
  
  if drop_duplicates:
    # Read and return the CSV file using the read_csv method
    print(f"Shape before remove duplicates: {df.shape}")

    # Use the drop_duplicated method to drop duplicates rows
    df = df.drop_duplicates(subset="text")

    print(f"Shape after remove duplicates: {df.shape}")

    if lower:
      # apply the str.lower() method to each element in the dataframe
      df = df.applymap(str.lower)
  return df

In [198]:
# Set the path to data
path_csv = "/content/Topic-Modeling-Reclame-Aqui/corpus.csv"

df = read_data(path_csv)

# Print the first 5 rows of the DataFrame
df.head(5)

Shape before remove duplicates: (12760, 2)
Shape after remove duplicates: (10510, 2)


,title,text
0,pedido cancelado sem justificativa após uma se...,eu estava pesquisando bastante uma nova tv par...
1,pedido cancelado,eu sinceramente estou decepcionada com o amazo...
2,cobrança indevida,cancelei meu plano antes de terminar o período...
3,pedido reincidente,"olha fiz compra veio errada, e veio errado nov..."
4,assinatura para vender na amazon brasil,eu me inscrevi na amazon para realizar vendas ...


This dataset contains just two columns called title and text 

In [199]:
print(df.shape)

(10510, 2)


There are 10510 unique rows in this dataset.

In [271]:
# join columns
df.columns = ["title", "documents"] 

# Use the replace() method to replace the string with an empty string
df = df.replace(re.compile('\[editado pelo reclame aqui\]|editado pelo reclame aqui|Editado pelo Reclame Aqui'), '')
df = df.replace(re.compile('\[casas bahia\]|Casa Bahia|Casas Bahia|casa bahia'), '')
df = df.replace(re.compile('\[magazine luiza\]|Magazine luiza|Magazine Luiza| Magazine luizar|Magazine Luizar'), '')
df = df.replace(re.compile('\[mercado livre\]|Mercado Livre|Mercado livre'), '')
df = df.replace(re.compile('\[americana\]|Ameriacanas|ameriacanas'), '')

# Drop the old index column
df.reset_index(inplace = True, drop = True)

df.head()

,title,documents
0,pedido cancelado sem justificativa após uma se...,eu estava pesquisando bastante uma nova tv par...
1,pedido cancelado,eu sinceramente estou decepcionada com o amazo...
2,cobrança indevida,cancelei meu plano antes de terminar o período...
3,pedido reincidente,"olha fiz compra veio errada, e veio errado nov..."
4,assinatura para vender na amazon brasil,eu me inscrevi na amazon para realizar vendas ...


#### **Save table as image**

In [253]:
# Set the path to save 
path = '/content/Topic-Modeling-Reclame-Aqui/results/table/'

# Use makedirs() to create a new directory if it does not exists
if not os.path.exists(path):
  os.makedirs(path)

# Save the DataFrame to a CSV file
df.head(10).to_csv(path + 'table.csv')

### **Preprocessing**

#### **Tokenization**

Tokenization aims to breaking text down into its component parts

In [202]:
WORD_TOKENIZER = nltk.tokenize.word_tokenize
def tokenize(text, lowercase=True):
  if lowercase:
    text = text.lower()
  return [token.strip() for token in WORD_TOKENIZER(text, language="portuguese")]

#### **Stem** 

Stem the tokens. This step aims to remove morphological affixes and normalize to standardized stem forms

In [203]:
STEMMER = nltk.stem.RSLPStemmer()
def stem(tokens):
  return [STEMMER.stem(token) for token in tokens]

#### **Lemmatize**

Lemmatize the tokens. Retains more natural forms than stemming, but assumes all tokens nons unless tokens are passed as (word, pos) tuples. Note: nltk lemmatize does not suport portugues language

In [204]:
LEMMATIZER = nltk.WordNetLemmatizer()

def lemmatize(tokens):
  lemmas = []
  for token in tokens:
      if isinstance(token, str):
          # treats token like a noun
          lemmas.append(LEMMATIZER.lemmatize(token)) 
      else: 
          # assume a tuple of (word, pos)
          lemmas.append(LEMMATIZER.lemmatize(*token))
  return lemmas

**Lemmatize option for portuguese text**

In [205]:
def lemmatize_pt(tokens, nlp):
  # Create a spaCy Doc object and apply the lemmatization
  doc = nlp(' '.join(tokens))

  # Return lemmatize
  return [token.lemma_ for token in doc]

#### **Remove stopwords**

Stop words are things like articles and conjunctions that usually do not offer a lot of value in an analysis.

In [290]:
custom_stop_words = ['amazon', 'americanas', 'casas bahia', 'magazine luiza', 'shein', 'kabum',
                       'samsung', 'mercado livre', 'banco brasil', 'apple', 'eletrolux', 'magazine', 'luiza', 'luizar',
                      'casas', 'bahia', 'casa', 'mercado', 'livre']

def remove_stopwords(tokens, stopwords=None, custom_stop_words = []):

  # Use the default stop words if none is passed
  if stopwords is None:
    stopwords = nltk.corpus.stopwords.words('portuguese') + custom_stop_words
  
  # Filter the list of tokens to exclude the stop word tokens
  return [token for token in tokens if token not in stopwords]

In [207]:
assert remove_stopwords(['casas bahia', 'amazon', 'roupa'], custom_stop_words=custom_stop_words) == ['roupa']

#### **Remove hyperlinks**

Removes http/s links from the tokens.

In [208]:
def remove_links(tokens):
  # Filter tokens that starts with "http://" or "https://"
  return [token for token in tokens 
          if not token.startswith("http://")
          and not token.startswith("https://")]

In [209]:
assert remove_links(['bom', 'http://online', 'https://offline']) == ['bom']

#### **Remove numbers**

In [302]:
def remove_numbers(tokens):
  # Use a regular expression to match words that contain numbers
  pattern = r"\b\w*\d\w*\b"
  tokens = [token for token in tokens if not re.sub(pattern, "", token) == ""]
  
  # Filter out number tokens using a list comprehension and the isnumeric method
  return [token for token in tokens if not token.isnumeric()]

In [211]:
assert remove_numbers(['ola', 'bicicleta', '1', '2002']) == ['ola', 'bicicleta']

#### **Remove date**

In [212]:
def remove_date(tokens):
  # Compile a regular expression to match dates in the format dd/mm or dd/mm/yyyy
  date_regex = re.compile(r'\d{2}/\d{2}(/\d{4})?')

  # Use the regex to find all the tokens that match the date pattern
  dates = [token for token in tokens if date_regex.fullmatch(token)]

  # Filter the list of tokens to exclude the date tokens
  filtered_tokens = [token for token in tokens if token not in dates]

  # Return the filtered tokens
  return filtered_tokens

In [213]:
assert remove_date(['texto', 'data', '20/10', 'seguro', '02/09/2014']) == ['texto', 'data', 'seguro']

#### **Remove punctuation**

In [214]:
def remove_punctuation(tokens,
                       strip_mentions=True,
                       strip_hashtags=True):

    tokens = [re.sub(r'[^\w\s]', '', token) for token in tokens]

    # Filter punctuation tokens
    tokens = [token.strip() for token in tokens if token not in string.punctuation]

    # Remove @ symbol from left side of tokens
    if strip_mentions:
        tokens = [t.lstrip(r"([!\"#$%&'()*+,-./:;<=>?@[\]^_`{|}~])\1+") for t in tokens]

    # Remove # symbol from left side of tokens
    if strip_hashtags:
        tokens = [t.lstrip(r"([!\"#$%&'()*+,-./:;<=>?@[\]^_`{|}~])\1+") for t in tokens]

    return tokens

In [215]:
assert remove_punctuation(['limpo', 'acento/  ///', 'simples???', 'onde', ',']) == ['limpo', 'acento', 'simples', 'onde']

#### **Remove short tokens**

In [216]:
def remove_short_tokens(tokens):
  # Filter the list of tokens to exclude tokens that are shorter than four letters
  filtered_tokens = [token for token in tokens if len(token) >= 4]

  # Return the filtered tokens
  return filtered_tokens

In [217]:
assert remove_short_tokens(['sair', 'um', 'correto', 'igual', 'oi']) == ['sair', 'correto', 'igual']

#### **Correction of spelling errors**

In [218]:
def check_spell_errors(text, spell):

  # Correct the spelling errors in the text
  corrected_text = spell.correction(text)

  # If no correction is present user the original text
  if corrected_text == None:
     corrected_text =  text
  
  # Return the corrected text
  return corrected_text

In [291]:
def preprocessing(documents, nlp = None, spell = None):
  corpus = []

  # process each document and append to corpus list
  for i, text in enumerate(documents):
    if i % 1000 == 0:
      print(f"{i} documents of {len(documents)}\n")
    if spell is not None:
      text = check_spell_errors(text, spell)
    tokens = tokenize(text)
    tokens = remove_links(tokens)
    tokens = remove_punctuation(tokens, strip_mentions=True, strip_hashtags=True)
    tokens = remove_numbers(tokens)
    tokens = remove_date(tokens)
    if nlp is not None: 
      tokens = lemmatize_pt(tokens, nlp)
    #tokens = stem(tokens) 
    tokens = remove_short_tokens(tokens)
    tokens = remove_stopwords(tokens,custom_stop_words=custom_stop_words)
    corpus.append(' '.join(tokens))
  return corpus

In [292]:
# Create a SpellChecker object
spell = SpellChecker(language='pt')

nlp = spacy.load('pt_core_news_sm')

corpus = preprocessing(df.documents, nlp, spell)

0 documents of 10510

1000 documents of 10510

2000 documents of 10510

3000 documents of 10510

4000 documents of 10510

5000 documents of 10510

6000 documents of 10510

7000 documents of 10510

8000 documents of 10510

9000 documents of 10510

10000 documents of 10510



In [221]:
# Print the first document before and after pre-processing it
print('Before preprocessing')
print(df.documents[0])
print('\nAfter preprocessig')
print(corpus[0])

Before preprocessing
eu estava pesquisando bastante uma nova tv para comprar e resolvi aguardar a semana do cliente, porque como ocorreu tiveram vários descontos e promoções nessa semana, sendo assim recebi descontos, cashback e cupons de varias plataformas e assim decidi efetuar a compra aquela que eu entendesse ser o melhor custo beneficio. sendo assim no dia 12 de setembro de 2022 recebi uma oferta de produto da amazon que entendi estar com um preço muito bom além de ter cachback e efetuei a compra que já aguardei até a semana do consumidor para efetuar com toda a expectativa. como estavamos anciosos e mesmo estando dentro do prazo de entrega hoje no dia 19/09 resolvi enviar uma mensagem perguntando quando o pedido seria enviado, ja que após a confirmação da compra e pagamento, passado 7 dias não tive nenhum retorno, já que complei inclusive na amazon, um site em que confio e tambem sou assinante. para a minha surpresa após o pedido de informação do pedido, tenho um retorno por emai

## **Training a BERTopic Model**

The BERTopic algorithm has several advantages over other topic modeling algorithms. It is able to handle sparse data, it is scalable to large datasets, and it is able to learn topics that are not well-defined or are overlapping.

As our data language is portuguese we will going to set language to multilingual.

Create a new BERTopic model and train it. By default BERTopic use the paraphrase-multilingual-MiniLM-L12-v2 model for multi language documents. For others model check here [BERTopic sentence transformers](https://maartengr.github.io/BERTopic/getting_started/embeddings/embeddings.html#sentence-transformers)

In [294]:
# reduce the impact of frequent words.
ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)

# Create a new BERTopic model using multilingual option
topic_model = BERTopic(language="multilingual", calculate_probabilities=True, verbose=True, ctfidf_model=ctfidf_model)

# Train model 
topics, probs = topic_model.fit_transform(corpus)

Batches:   0%|          | 0/329 [00:00<?, ?it/s]

2023-01-05 04:24:32,711 - BERTopic - Transformed documents to Embeddings
2023-01-05 04:24:52,185 - BERTopic - Reduced dimensionality
2023-01-05 04:24:57,381 - BERTopic - Clustered reduced embeddings


BERTopic works in three main steps: 


1.   Documents are first converted to numeric data. It extracts different embeddings based on the context of the word. For this, a sentence transformation model is used.
2.  Documents with similar topics are then grouped together forming clusters with similar topics. For this purpose, BERTopic uses the clustering algorithm UMAP to lower the dimensionality of the embeddings. Then the documents are clustered with the density-based algorithm HDBSCAN.
3. BERTopic extracts topics from clusters using a class-based TF-IDF score. This score gives the importance of each word in a cluster. Topics are then created based on the most important words measured by their C-TF-IDF score.

For more information check this link [BERTopic](https://towardsdatascience.com/topic-modeling-with-bert-779f7db187e6)



### **Extracting Topics**

In [295]:
# Print the most frequent topics
freq = topic_model.get_topic_info()

# Show the top 5 most frequent topics
freq.head(5)

,Topic,Count,Name
0,-1,4764,-1_dinheiro_responder_reclamação_transportadora
1,0,587,0_cartão_crédito_cobrança_cobrar
2,1,361,1_celular_smartphone_aparelho_fone
3,2,328,2_remetente_endereço_rastreio_correio
4,3,268,3_cancelamento_cancelar_cancelei_solicitei


The table above shows the five most freqeuente topics and the words present on it extract by BERTopic. -1 refers to all outliers and should be ignored.

In [296]:
# show the most frequent topic
topic_model.get_topic(1)

[('celular', 0.3070375345763583),
 ('smartphone', 0.23022531059907597),
 ('aparelho', 0.20786430349498292),
 ('fone', 0.17090118701044152),
 ('motorola', 0.14384204339109585),
 ('chip', 0.14223872438506316),
 ('garantia', 0.14024767389278864),
 ('xiaomi', 0.1378048870129252),
 ('telefone', 0.1369942853495296),
 ('redmi', 0.12676562409147094)]

**Note:** BERTopic is stocastich which means that the topics might differ across runs this is mostly due to the stocastisch nature of UMAP

#### **Save topic info table as CSV**

In [297]:
# Set the path to save 
path = '/content/Topic-Modeling-Reclame-Aqui/results/topic_info_tables/'

# Use makedirs() to create a new directory if it does not exists
if not os.path.exists(path):
  os.makedirs(path)

# Save table as csv
freq.head(10).to_csv(path + 'topic_info_preprocessed_lemma.csv')

## **Visualization**

### **Intertopic Distance Map**

This graph shows the distance intertopic and help us understand the promixity of topics

In [298]:
fig = topic_model.visualize_topics()
fig

#### **Save intertopic distance map**

In [299]:
# Set the path to save 
path = '/content/Topic-Modeling-Reclame-Aqui/results/intertopic_distance_map/'

# Use makedirs() to create a new directory if it does not exists
if not os.path.exists(path):
  os.makedirs(path)


fig.write_image(path + "idm_preprocessed_lemma.png", format="png")

[Github link](https://github.com/punkmic/Topic-Modeling-Reclame-Aqui/blob/master/results/intertopic_distance_map/idm_preprocessed_lemma.png?raw=true)

### **Visualize Topic Hierarchy**

The topics that were created can be hierarchically reduced. This visualization shows how the topics relate to one another.

In [300]:
fig = topic_model.visualize_hierarchy(top_n_topics=30)
fig

#### **Save Hierarchical Clustering**

In [280]:
# Set the path to save 
path = '/content/Topic-Modeling-Reclame-Aqui/results/hierarchical_clustering/'

# Use makedirs() to create a new directory if it does not exists
if not os.path.exists(path):
  os.makedirs(path)


fig.write_image(path + "hc_preprocessed_lemma.png", format="png")

[Github link](https://github.com/punkmic/Topic-Modeling-Reclame-Aqui/blob/master/results/hierarchical_clustering/hc_preprocessed_lemma.png?raw=true)

### **Visualize Terms**

We can visualize the selected terms for a few topics by creating bar charts out of the c-TF-IDF scores for each topic representation.

In [301]:
fig = topic_model.visualize_barchart(top_n_topics=12)
fig

#### **Save Top Word Scores Bar Chart**

In [282]:
# Set the path to save 
path = '/content/Topic-Modeling-Reclame-Aqui/results/top_words_scores/'

# Use makedirs() to create a new directory if it does not exists
if not os.path.exists(path):
  os.makedirs(path)


fig.write_image(path + "tws_preprocessed_lemma.png", format="png")

[Github link](https://github.com/punkmic/Topic-Modeling-Reclame-Aqui/blob/master/results/top_words_scores/tws_preprocessed_lemma.png?raw=true)

### **Visualize Topic Similarity**

This plot shows a similarity matrix by simply applying cosine similarities through those topic embeddings generate by BERTopic through both c-TF-IDF and embeddings. This matrix indicate how similar certain topics are to each other.

In [283]:
fig = topic_model.visualize_heatmap(n_clusters=20, width=1000, height=1000)
fig

#### **Save Similarity Matrix**

In [284]:
# Set the path to save 
path = '/content/Topic-Modeling-Reclame-Aqui/results/similarity_matrix/'

# Use makedirs() to create a new directory if it does not exists
if not os.path.exists(path):
  os.makedirs(path)


fig.write_image(path + "sm_preprocessed_lemma.png", format="png")

[Github link](https://github.com/punkmic/Topic-Modeling-Reclame-Aqui/blob/master/results/similarity_matrix/sm_preprocessed_lemma.png?raw=true)

### **Visualize Term Score Decline**

Topics are represented by a number of words starting with the best representative word. Each word is represented by a c-TF-IDF score. The higher the score, the more representative a word to the topic is. Since the topic words are sorted by their c-TF-IDF score, the scores slowly decline with each word that is added.

In [285]:
fig = topic_model.visualize_term_rank()
fig

#### **Save Term score decline per Topic**

In [286]:
# Set the path to save 
path = '/content/Topic-Modeling-Reclame-Aqui/results/term_score_decline_topic/'

# Use makedirs() to create a new directory if it does not exists
if not os.path.exists(path):
  os.makedirs(path)


fig.write_image(path + "tsdp_preprocessed_lemma.png", format="png")

[Github link](https://github.com/punkmic/Topic-Modeling-Reclame-Aqui/blob/master/results/term_socore_decline_topic/tsdp_preprocessed_lemma.png?raw=true)

### **Term search**

In [287]:
# Find topics that contains blackfriday term
similar_topics, similarity = topic_model.find_topics("blackfriday", top_n=5)

# Show similar topics
similar_topics

[24, 71, 75, 46, 44]

In [288]:
# Show a specific topic
topic_model.get_topic(34)

[('china', 0.11533061824681948),
 ('nacional', 0.031122997189302736),
 ('correio', 0.02551847519913509),
 ('encomenda', 0.022894803917738172),
 ('atualizar', 0.021179338471183872),
 ('pedido', 0.019306669862288222),
 ('chegar', 0.018046828679699985),
 ('saber', 0.017934439628813588),
 ('local', 0.01761753023913906),
 ('postar', 0.01659683681921582)]

### **Hiperparameter optimization**

In [238]:
%%capture
!pip install optuna
!pip install hdbscan
!pip install umap-learn

In [239]:
import json
import optuna
from hdbscan import HDBSCAN
from umap import UMAP
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.vectorizers import ClassTfidfTransformer
import ast

In [240]:
def save_documents(model, corpus, trial, reduced):
  fig = topic_model.visualize_documents(corpus, hide_document_hover=True, hide_annotations=True)

  # Set the path to save 
  path = '/content/Topic-Modeling-Reclame-Aqui/results/documents_n_topics/'

  if reduced:
    path = path + 'reduced/'

  # Use makedirs() to create a new directory if it does not exists
  if not os.path.exists(path):
    os.makedirs(path)

  fig.write_image(path + f"document_n_topics_trial_{trial}.png", format="png")

In [241]:
def save_topics(model, trial, reduced=False):
  fig = model.visualize_topics()

  # Set the path to save 
  path = '/content/Topic-Modeling-Reclame-Aqui/results/intertopic_distance_map/'

  if reduced:
    path = path + 'reduced/'

  # Use makedirs() to create a new directory if it does not exists
  if not os.path.exists(path):
    os.makedirs(path)

  fig.write_image(path + f"intertopic_distance_map_trial_{trial}.png", format="png")

In [242]:
def save_hierarchy(model, trial, reduced=False):
  fig = model.visualize_hierarchy()

  # Set the path to save 
  path = '/content/Topic-Modeling-Reclame-Aqui/results/hierarchical_clustering/'

  if reduced:
    path = path + 'reduced/'

  # Use makedirs() to create a new directory if it does not exists
  if not os.path.exists(path):
    os.makedirs(path)

  fig.write_image(path + f"hierarchical_clustering_trial_{trial}.png", format="png")

In [243]:
def save_top_words_scores(model, trial, reduced=False):
  fig = model.visualize_barchart()

  # Set the path to save 
  path = '/content/Topic-Modeling-Reclame-Aqui/results/top_words_scores/'

  if reduced:
    path = path + 'reduced/'

  # Use makedirs() to create a new directory if it does not exists
  if not os.path.exists(path):
    os.makedirs(path)

  fig.write_image(path + f"top_words_scores_trial_{trial}.png", format="png")

In [244]:
def save_similarity_matrix(model, trial, reduced=False):
  fig = model.visualize_heatmap()

  # Set the path to save 
  path = '/content/Topic-Modeling-Reclame-Aqui/results/similarity_matrix/'

  if reduced:
    path = path + 'reduced/'

  # Use makedirs() to create a new directory if it does not exists
  if not os.path.exists(path):
    os.makedirs(path)

  fig.write_image(path + f"similarity_matrix_trial_{trial}.png", format="png")

In [245]:
def save_term_rank(model, trial, reduced=False):
  fig = model.visualize_term_rank()

  # Set the path to save 
  path = '/content/Topic-Modeling-Reclame-Aqui/results/term_score_decline_topic/'

  if reduced:
    path = path + 'reduced/'

  # Use makedirs() to create a new directory if it does not exists
  if not os.path.exists(path):
    os.makedirs(path)

  fig.write_image(path + f"term_score_trial_{trial}.png", format="png")

In [246]:
def save_hyperparameters(trial_params, label):
  # Set the path to save 
  path = '/content/Topic-Modeling-Reclame-Aqui/results/hyperparameters/'

  # Use makedirs() to create a new directory if it does not exists
  if not os.path.exists(path):
    os.makedirs(path)

  with open(path + f"hyperparameters_trial_{label}.json", "w") as f:
    f.write(json.dumps(trial_params))


In [306]:
def optimizer(trial):

  # BERTopic hyperparameters
  top_n_words = trial.suggest_int('bertopic__top_n_words', 5, 10)
  n_gram_range = ast.literal_eval(trial.suggest_categorical('bertopic__n_gram_range', ['(1,1)', '(1,2)', '(1,3)']))
  min_topic_size = trial.suggest_int('bertopic__min_topic_size', 15, 20)
  diversity = trial.suggest_int('bertopic__diversity', 0.1, 0.9)
  outlier_threshold = trial.suggest_float('bertopic__outliers_threshold', 0.04, 0.09)
  nr_topics = trial.suggest_int('bertopic__nr_topics', 12, 20)

  # UMAP hyperparameters
  n_neighbors = trial.suggest_int('umap__n_neighbors', 10, 20)
  n_components = trial.suggest_int('umap__n_components', 2, 6)
  metric = trial.suggest_categorical('umap__metric', ['cosine', 'euclidean'])
  min_dist = trial.suggest_float('umap__min_dist', 0.1, 1.0)
  spread = trial.suggest_float('umap__spread', 0.1, 1.0)

  # HDBSCAN hyperprarameters
  min_cluster_size = trial.suggest_int('hdbscan__min_cluster_size', 10, 20)
  min_samples = trial.suggest_int('hdbscan__min_samples', 10, 20)
  cluster_selection_epsilon = trial.suggest_float('hdbscan__cluster_selection_epsilon', 0.1, 1.0)

  # CountVectorizer hyperparameters 
  max_features = trial.suggest_int('vectorizer__max_features', 3000, 6000)

  # reduce the impact of frequent words.
  ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)

  stop_words = nltk.corpus.stopwords.words('portuguese') + custom_stop_words
  vectorizer_model = CountVectorizer(stop_words=stop_words, ngram_range=n_gram_range, max_features=max_features)

  umap_model = UMAP(n_neighbors=n_neighbors, metric=metric, random_state=42)

  hdbscan_model = HDBSCAN(min_cluster_size=min_cluster_size, prediction_data=True)

  # create a new BERTopic model using multilingual option
  model = BERTopic(language="multilingual", 
                         calculate_probabilities=True, 
                         verbose=False,
                         nr_topics='auto',
                         top_n_words=top_n_words,
                         n_gram_range=n_gram_range,
                         min_topic_size=min_topic_size,
                         diversity=diversity,
                         umap_model=umap_model,
                         hdbscan_model=hdbscan_model,
                         vectorizer_model=vectorizer_model,
                         ctfidf_model=ctfidf_model)
  
  label = trial.number
  params = trial.params

  # train model 
  topics, probs = model.fit_transform(corpus)

  # save plots
  save_topics(model, label, False)
  save_documents(model, corpus, label, False)
  save_hierarchy(model, label, False)
  #save_similarity_matrix(model, label, False)
  save_term_rank(model, label, False)
  save_top_words_scores(model, label, False)

  # reduce outliers
  new_topics = model.reduce_outliers(corpus,
                                     topics, 
                                     probabilities=probs,
                                     strategy="probabilities",
                                     threshold=outlier_threshold)
  model.update_topics(corpus, topics=new_topics)

  # save plots after reducion
  save_topics(model, label, True)
  save_documents(model, corpus, label, True)
  save_hierarchy(model, label, True)
  #save_similarity_matrix(model, label, True)
  save_term_rank(model, label, True)
  save_top_words_scores(model, label, True)

  # save hyperparameters
  save_hyperparameters(params, label)

  # save model
  path = '/content/Topic-Modeling-Reclame-Aqui/results/models/'

  # Use makedirs() to create a new directory if it does not exists
  if not os.path.exists(path):
    os.makedirs(path)

  model.save(path + f"model_trial_{label}")

  return 0

In [307]:
%%time
study = optuna.create_study(study_name='Bertopic')
study.optimize(optimizer, n_trials=2)

[I 2023-01-05 04:40:19,493] A new study created in memory with name: Bertopic
2023-01-05 04:40:37,903 - BERTopic - Transformed documents to Embeddings
2023-01-05 04:40:52,497 - BERTopic - Reduced dimensionality
2023-01-05 04:40:54,519 - BERTopic - Clustered reduced embeddings
2023-01-05 04:40:58,673 - BERTopic - Reduced number of topics from 38 to 28
[I 2023-01-05 04:42:09,357] Trial 0 finished with value: 0.0 and parameters: {'bertopic__top_n_words': 7, 'bertopic__n_gram_range': '(1,1)', 'bertopic__min_topic_size': 20, 'bertopic__diversity': 0, 'bertopic__outliers_threshold': 0.057618081716460476, 'bertopic__nr_topics': 19, 'umap__n_neighbors': 20, 'umap__n_components': 2, 'umap__metric': 'cosine', 'umap__min_dist': 0.5932562838223142, 'umap__spread': 0.36796133928601804, 'hdbscan__min_cluster_size': 12, 'hdbscan__min_samples': 20, 'hdbscan__cluster_selection_epsilon': 0.36029414774524793, 'vectorizer__max_features': 4857}. Best is trial 0 with value: 0.0.
2023-01-05 04:42:29,527 - BE

CPU times: user 4min 23s, sys: 10.6 s, total: 4min 33s
Wall time: 4min 11s


#### **Merge of similar topics**

In [249]:
# TODO: select the best model and merge similar topic and save final model

#topic_model = BERTopic.load("best_model")

In [250]:
##########33!rm -rf /content/Topic-Modeling-Reclame-Aqui/results